In [1]:
import pandas as pd
import urllib
from urllib.request import Request, urlopen
import requests
from bs4 import BeautifulSoup, Comment
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
from imageai.Prediction.Custom import CustomImagePrediction
import os

/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/rahullat/.pyenv/versions/3.6.5

In [2]:
execution_path = os.getcwd()
prediction = CustomImagePrediction()
prediction.setModelTypeAsResNet()
prediction.setModelPath("model_ex-088_acc-0.918269.h5")
prediction.setJsonPath("model_class.json")
prediction.loadModel(num_objects=2)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [28]:
df = pd.read_csv('finalData.csv')
df["Catagory"] = df["team_name"].str.split().str[-1]
warriors = df[df["Catagory"] == 'Warriors']

In [56]:
for index, row in warriors.iterrows():
    citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    soup = BeautifulSoup(req.text)
    try:
        image = soup.find('img', {"id": "ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Image"})['src']
        print(image)
        nu_name = "wi/" + row.team_name + row.city + ".gif"
        urllib.request.urlretrieve(image, nu_name)
        probabilities = prediction.predictImage(nu_name, result_count=2)
        if probabilities[1][1] > 3:
            is_racist.append('notRacist')
            not_racist_percent.append(probabilities[1][1])
            print(row.team_name + str(probabilities[1]) + " notRacist")
        elif probabilities[1][1] > 0.1:
            is_racist.append('check')
            not_racist_percent.append(probabilities[1][1])
            print(row.team_name + str(probabilities[1]) + " check")
        else:
            is_racist.append('racist')
            not_racist_percent.append(probabilities[1][1])
            print(row.team_name + str(probabilities[1]) + " racist")
    except:
        print(row.team_name + " not_there")
        is_racist.append('not_there')
        not_racist_percent.append('')
        continue

https://dw3jhbqsbya58.cloudfront.net/fit-in/120x/school-mascot/3/f/8/3f82ec6b-488c-412a-8b5c-3833173e6661.gif?version=636252583200000000
Indian Land Warriors[97.90798425674438, 2.0920155569911003] check
https://dw3jhbqsbya58.cloudfront.net/team-logo/2e94d997-19fb-4831-a0d9-6be6f3784791_boys_football.gif
Indian River Warriors[74.14278388023376, 25.857213139533997] notRacist
Indianola Warriors not_there
St. Francis Indian Warriors not_there
Abundant Life Academy Warriors not_there
https://dw3jhbqsbya58.cloudfront.net/fit-in/120x/school-mascot/9/0/5/905df7e5-7ba9-4243-b29b-5dcb0e3a908e.gif?version=636739027800000000
Adair Warriors[99.60505366325378, 0.3949505742639303] check
https://dw3jhbqsbya58.cloudfront.net/fit-in/120x/school-mascot/6/d/1/6d1b1f51-655c-46df-8a72-af8ddfd99749.gif?version=636077134800000000
Adena Warriors[98.41179847717285, 1.5881970524787903] check
Ahoskie Christian Lady Warriors not_there
Ahoskie Christian Warriors not_there
Alamance Christian Warriors not_there
https

In [69]:
len(is_racist)# warriors['not_racist_percent'] = not_racist_percent
res = is_racist[-689:] 
warriors['is_racist'] = res

/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [71]:
warriors_check = warriors[warriors['is_racist'] == 'notRacist']
warriors_check.to_csv('warriorsCheck.csv')

In [13]:
warriors = warriors[warriors['is_racist'] != 'notRacist']

In [15]:
warriors = warriors[warriors['is_racist'] != 'not_there']

In [24]:
check = warriors[warriors['is_racist'] == 'check']
racistWarriors = warriors[warriors['is_racist'] == 'racist']

In [21]:
df = df[df['Catagory']!='Warriors']
df

,Unnamed: 0,Unnamed: 0.1,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,Catagory
0,0,3,Ackerman Indians,Ackerman,Mississippi,active,/teams/ackerman-indians,NaN,33.310124,-89.172843,MS,True,Indians
1,1,7,Adair County Indians,Columbia,Kentucky,active,/teams/adair-county-indians,NaN,37.102841,-85.306350,KY,True,Indians
2,3,39,Altamont Indians,Altamont,Illinois,active,/teams/altamont-indians,NaN,39.062036,-88.748122,IL,True,Indians
3,4,44,Alvarado Indians,Alvarado,Texas,active,/teams/alvarado-indians,NaN,32.409557,-97.210288,TX,True,Indians
4,5,48,Amesbury Indians,Amesbury,Massachusetts,active,/teams/amesbury-indians,NaN,42.857954,-70.930092,MA,True,Indians
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1375,109,2412,Tulsa Central Braves,Tulsa,Oklahoma,active,/teams/tulsa-central-braves,NaN,36.155681,-95.992911,OK,True,Braves
1376,110,785,Foothills Academy College Preparatory Blackhawks,Scottsdale,Arizona,active,/teams/foothills-academy-college-preparatory-b...,NaN,33.494219,-111.926018,AZ,True,Blackhawks
1377,115,1085,James Caldwell Chiefs,West Caldwell,New Jersey,active,/teams/james-caldwell-chiefs,NaN,40.840933,-74.301815,NJ,True,Chiefs
1378,116,1590,Nonnewaug Valley Chiefs,Woodbury,Connecticut,active,/teams/nonnewaugshepaug-valley-chiefs,NaN,41.544540,-73.209002,CT,True,Chiefs


,Unnamed: 0,Unnamed: 0.1,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,Catagory,is_racist,not_racist_percent
149,158,1039,Indian Land Warriors,Fort Mill,South Carolina,active,/teams/indian-land-warriors,NaN,35.007370,-80.945076,SC,True,Warriors,check,2.09202
484,515,8,Adair Warriors,Adair,Oklahoma,active,/teams/adair-warriors,NaN,35.878314,-94.656740,OK,True,Warriors,check,0.394951
485,516,9,Adena Warriors,Frankfort,Ohio,active,/teams/adena-warriors,NaN,39.401452,-83.180743,OH,True,Warriors,check,1.5882
489,520,24,Albany Warriors,Albany,Missouri,active,/teams/albany-warriors,NaN,40.248607,-94.331068,MO,True,Warriors,check,0.18284
491,522,33,Aloha Warriors,Beaverton,Oregon,active,/teams/aloha-warriors,NaN,45.487172,-122.803780,OR,True,Warriors,check,1.9524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1111,1218,2789,Woodlan Warriors,Woodburn,Indiana,active,/teams/woodlan-warriors,NaN,41.125327,-84.853296,IN,True,Warriors,check,0.193343
1118,1226,2804,Worland Warriors,Worland,Wyoming,active,/teams/worland-warriors,NaN,44.016833,-107.955797,WY,True,Warriors,check,0.947149
1119,1227,2805,Worthington Christian Warriors,Worthington,Ohio,active,/teams/worthington-christian-warriors,NaN,40.093094,-83.017959,OH,True,Warriors,check,1.74561
1124,1233,2812,Wyoming East Warriors,New Richmond,West Virginia,active,/teams/wyoming-east-warriors,NaN,37.570113,-81.488715,WV,True,Warriors,check,0.629103


In [26]:
imageurls = []
for index, row in check.iterrows():
    citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    soup = BeautifulSoup(req.text)
    image = soup.find('img', {"id": "ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Image"})['src']
    imageurls.append(image)
    probabilities = prediction.predictImage(nu_name, result_count=2)

check['url'] = imageurls

/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [30]:
check.to_csv('checkWarriors.csv')

In [32]:
imageurls = []
for index, row in racistWarriors.iterrows():
    citystring = row.team_name.lower().replace(" ","-") + "-" + "(" + row.city.lower().replace(" ","-") + "," + row.abbrev.lower() + ")"
    url = 'https://www.maxpreps.com/high-schools/'+citystring+'/football/schedule.htm'
    hdr = {'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9'}
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    req = session.get(url, headers=hdr)
    soup = BeautifulSoup(req.text)
    image = soup.find('img', {"id": "ctl00_NavigationWithContentOverRelated_ContentOverRelated_PageHeaderUserControl_Image"})['src']
    imageurls.append(image)
    probabilities = prediction.predictImage(nu_name, result_count=2)

racistWarriors['url'] = imageurls
racistWarriors.to_csv('racists.csv')

/Users/rahullat/.pyenv/versions/3.6.5/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [33]:
checked = pd.read_csv('checkWarriors.csv')
racistWarriors = pd.read_csv('racists.csv')

In [36]:
checked = checked[checked['is_racist']=='racist']

In [41]:
checked = checked[['Unnamed: 0','team_name','city','state','status','url','tidbit','lat','long','abbrev','exists']]
racistWarriors = racistWarriors[['Unnamed: 0','team_name','city','state','status','url','tidbit','lat','long','abbrev','exists']]

In [47]:
df = df[df['Catagory']!='Warriors']
final_schools = pd.concat([df,checked,racistWarriors])

In [53]:
final_schools = final_schools.drop_duplicates()
final_schools["Catagory"] = final_schools["team_name"].str.split().str[-1]
final_schools = final_schools[final_schools["Catagory"] != 'Thunderbirds']
final_schools = final_schools[final_schools["Catagory"] != 'Raiders']
final_schools.to_csv('finalfinal.csv')

In [54]:
final_schools

,Unnamed: 0,Unnamed: 0.1,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,Catagory
0,0,3.0,Ackerman Indians,Ackerman,Mississippi,active,/teams/ackerman-indians,NaN,33.310124,-89.172843,MS,True,Indians
1,1,7.0,Adair County Indians,Columbia,Kentucky,active,/teams/adair-county-indians,NaN,37.102841,-85.306350,KY,True,Indians
2,3,39.0,Altamont Indians,Altamont,Illinois,active,/teams/altamont-indians,NaN,39.062036,-88.748122,IL,True,Indians
3,4,44.0,Alvarado Indians,Alvarado,Texas,active,/teams/alvarado-indians,NaN,32.409557,-97.210288,TX,True,Indians
4,5,48.0,Amesbury Indians,Amesbury,Massachusetts,active,/teams/amesbury-indians,NaN,42.857954,-70.930092,MA,True,Indians
...,...,...,...,...,...,...,...,...,...,...,...,...,...
46,1030,NaN,Wes-Del Warriors,Gaston,Indiana,active,https://dw3jhbqsbya58.cloudfront.net/fit-in/12...,NaN,40.313931,-85.500528,IN,True,Warriors
47,1034,NaN,West Caldwell Warriors,Lenoir,North Carolina,active,https://dw3jhbqsbya58.cloudfront.net/fit-in/12...,NaN,35.259037,-77.648488,NC,True,Warriors
48,1062,NaN,Westside Warriors,Jonesboro,Arkansas,active,https://dw3jhbqsbya58.cloudfront.net/fit-in/12...,NaN,35.842865,-90.703452,AR,True,Warriors
49,1086,NaN,Wildwood Warriors,Wildwood,New Jersey,active,https://dw3jhbqsbya58.cloudfront.net/fit-in/12...,NaN,38.986292,-74.821724,NJ,True,Warriors


In [55]:
warriors

,Unnamed: 0,Unnamed: 0.1,team_name,city,state,status,url,tidbit,lat,long,abbrev,exists,Catagory
149,158,1039,Indian Land Warriors,Fort Mill,South Carolina,active,/teams/indian-land-warriors,NaN,35.007370,-80.945076,SC,True,Warriors
152,162,1043,Indian River Warriors,Philadelphia,New York,active,/teams/indian-river-warriors,NaN,44.154622,-75.708847,NY,True,Warriors
157,168,1051,Indianola Warriors,Indianola,Oklahoma,active,/teams/indianola-warriors,NaN,35.162876,-95.774715,OK,True,Warriors
360,380,2255,St. Francis Indian Warriors,St. Francis,South Dakota,active,/teams/st-francis-indian-warriors,NaN,43.144450,-100.905147,SD,True,Warriors
483,514,2,Abundant Life Academy Warriors,Kanab,Utah,active,/teams/abundant-life-academy-warriors,NaN,37.048142,-112.528548,UT,True,Warriors
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,94,2582,Waxahachie Preparatory Academy Warriors,Waxahachie,Texas,active,/teams/waxahachie-preparatory-academy-warriors,NaN,32.394491,-96.843936,TX,True,Warriors
1367,97,2675,Westland Lutheran Warriors,Westland,Michigan,active,/teams/westland-lutheran-warriors,NaN,42.323806,-83.400532,MI,True,Warriors
1368,98,2705,Wheaton/Herman-Norcross Warriors,Wheaton,Minnesota,active,/teams/wheatonherman-norcross-warriors,NaN,45.804405,-96.499233,MN,True,Warriors
1369,100,2724,Whitney M. Young Warriors,Cleveland,Ohio,active,/teams/whitney-m-young-warriors,NaN,41.505161,-81.693445,OH,True,Warriors


In [59]:
warriors.to_csv('warriorsCheck.csv')

In [73]:
warriors_checked_fixed = pd.read_csv('warriorsCheck.csv')

In [74]:
warriors_checked_fixed = warriors_checked_fixed[warriors_checked_fixed['is_racist']=='racist']

In [75]:
warriors_checked_fixed = warriors_checked_fixed[['Unnamed: 0','team_name','city','state','status','url','tidbit','lat','long','abbrev','exists']]

In [78]:
df = df[df['Catagory']!='Warriors']

In [79]:
final_schools = pd.concat([df,checked,racistWarriors,warriors_checked_fixed])
final_schools = final_schools.drop_duplicates()
final_schools["Catagory"] = final_schools["team_name"].str.split().str[-1]
final_schools = final_schools[final_schools["Catagory"] != 'Thunderbirds']
final_schools = final_schools[final_schools["Catagory"] != 'Raiders']
final_schools.to_csv('finalfinal.csv')

In [ ]:
final_schools = pd.concat([df,checked,racistWarriors])